In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
model_name = "E:\\models\\pleasRAG"
token = os.getenv("HFTOKEN")
import torch

In [2]:
from transformers import AutoModelForCausalLM 
from transformers import AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\khatt\Documents\AmazonEcho-NLP-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    return_full_text=True
)

Device set to use cpu


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "E:/models/gte-small"
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    
)


C:\Users\khatt\AppData\Local\Temp\ipykernel_23764\2798243950.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [10]:
import sys
import os

# Go up one level (or more depending on where your .ipynb file is)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # adjust if needed
sys.path.append(project_root)
from WebsiteScrap import WebsiteScrapper
web = WebsiteScrapper("https://www.amazon.com/Amazon-Basics-Everyday-Plates-Disposable/dp/B0C2CY22B8/?_encoding=UTF8&pd_rd_w=APjaP&content-id=amzn1.sym.f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_p=f2128ffe-3407-4a64-95b5-696504f68ca1&pf_rd_r=YXMEXN435CNCQZWD512A&pd_rd_wg=7Izqy&pd_rd_r=3fbb0d02-8f22-4b11-ac45-f11f9ef282d6&ref_=pd_hp_d_btf_crs_zg_bs_284507&th=1")
dfname = web.websitename()

'AmazonBasicsEverydayPlatesDisposable'